# 2 – Training (Final)

Dieses Notebook trainiert für eine gegebene `EXP_ID` das Zwei-Stufen-
XGBoost-Modell (Signal + Richtung). Es liest die Config und den
Trainingsdatensatz aus `data/processed/...` und erzeugt nur den
Modell-Fit und einfache Metriken.


In [ ]:
# Bitte EXP_ID explizit setzen, passend zur Data-Prep.
EXP_ID = 'CHANGE_ME'  # z.B. 'v3_h4_thr0p3pct_relaxed'
assert EXP_ID != 'CHANGE_ME', 'Bitte EXP_ID oben setzen.'


In [ ]:
import json
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from src.utils.io import DATA_PROCESSED
from src.models.train_xgboost_two_stage import (
    split_train_val_test,
    build_signal_targets,
    build_direction_targets,
    train_xgb_binary,
    get_feature_cols,
)

plt.style.use('seaborn-v0_8')

# Datensatz laden
ds_path = DATA_PROCESSED / 'datasets' / f'eurusd_news_training__{EXP_ID}.csv'
print('Verwende Datensatz:', ds_path)
df = pd.read_csv(ds_path, parse_dates=['date']).sort_values('date').reset_index(drop=True)
print(df.shape)
feature_cols = get_feature_cols(df)
print('Feature-Spalten:', len(feature_cols))


In [ ]:
# Zeitliche Splits
test_start = '2025-01-01'
train_frac_pretest = 0.8

splits = split_train_val_test(
    df, pd.to_datetime(test_start), train_frac_within_pretest=train_frac_pretest
)
for name, split_df in splits.items():
    print(name, split_df['date'].min().date(), '->', split_df['date'].max().date(), 'n=', len(split_df))
